# Laboratorio 5

WWI (World Wide Importers) 1 es una empresa encargada de realizar importaciones y venderlas a diferentes clientes en diferentes ciudades de Estados Unidos. Actualmente, la empresa se encuentra buscando servicios de consultoría de BI puesto que desean optimizar sus ganancias, pues consideran que algunos de sus productos no están generando las ganancias que deberían. También, están interesados en saber si hay otros factores que le impiden optimizar sus ganancias. Dado lo anterior, WWI lo contrata a usted para que realice una consultoría de BI, en particular en esta fase, para la creación de la base de datos, la carga de datos y unas consultas iniciales que permitan validar el proceso previo. En esta ocasión quiere que el trabajo lo realicen en grupos de máximo 3 estudiantes, con el fin de comparar dos herramientas que está evaluando para utilizar a futuro en procesos de ETL.
Para esta etapa de la consultoría, la empresa requiere que usted implemente un proceso ETL que le permita extraer los datos de órdenes desde unos archivos CSV y almacenarlos en un modelo dimensional tal que les permita realizar análisis para mejorar entre otros elementos, su eficiencia operativa.

## Análisis exploratorio de los datos

In [1]:
#Instalamos las dependencias necesarias
!pip3 install pandas_profiling
!pip3 install pandas==1.2

In [ ]:
!pip3 show pandas

Se realizará el análisis exploratorio para cada uno de los datasets proveídos por WWI

In [ ]:
# Se importan las librerías y dependencias necesarias para la ejecución del notebook
import numpy as np
import pandas as pd
import matplotlib as plt
from pandas_profiling import ProfileReport

### Variable Dimension City

In [ ]:
datos_dim_city = pd.read_csv('datos_originales/dimension_city.csv', delimiter=',')
datos_dim_city.sample(5)

Observamos las primeras filas del dataset

In [ ]:
datos_dim_city.head()

Vemos que la primera fila, con row ID Row0, no aporta información relevante pues todos sus datos están ausentes. Procedemos entonces a eliminarla.

In [ ]:
datos_dim_city = datos_dim_city.drop(0)
#Verificamos que se eliminó correctamente
datos_dim_city.head()

Veamos, ahora, el número de filas y columnas que tenemos en nuestro conjunto de datos:

In [ ]:
print("Número de filas o datos: {}".format(len(datos_dim_city)))
print("Número de columnas o variables: {}".format(len(datos_dim_city.columns)))

Vemos que tenemos 98 datos (filas) y 10 columnas (variables).

Para ver una exploración más exhaustiva, hicimos un pandas profiling de los datos que nos fueron entregados:

In [ ]:
profiling = ProfileReport(datos_dim_city)
profiling.to_file(output_file="pandas_profiling/dimension_city_pandas_profiling.html")
profiling

Del anterior reporte podemos concluir y considerar:

- No encontamos filas duplicadas
- Las variables row ID y City_Key tienen un valor distinto en cada fila. Se podría considerar eliminar una de las dos variables.
- No encontramos celdas ausentes.
- Las variables Country, Continent, Region y Subregion tienen el mismo valor en todas las celdas. Esto indica que no aportan información relevante y pueden ser eliminadas.

### Variable Dimension Customer

In [ ]:
datos_dim_cust = pd.read_csv('datos_originales/dimension_customer.csv', delimiter=',', encoding='latin-1')
datos_dim_cust.head()

Vemos que la primera fila, con Customer_Key 0, no aporta información relevante pues todos sus datos están ausentes. Procedemos entonces a eliminarla.

In [ ]:
datos_dim_cust = datos_dim_cust.drop(0)
#Verificamos que se eliminó correctamente
datos_dim_cust.head()

Veamos, ahora, el número de filas y columnas que tenemos en nuestro conjunto de datos:

In [ ]:
print("Número de filas o datos: {}".format(len(datos_dim_cust)))

print("Número de columnas o variables: {}".format(len(datos_dim_cust.columns)))

Para ver una exploración más exhaustiva, hicimos un pandas profiling de los datos que nos fueron entregados:

In [ ]:
profiling = ProfileReport(datos_dim_cust)
profiling.to_file(output_file="pandas_profiling/dimension_customer_pandas_profiling.html")
profiling

Del anterior reporte podemos concluir y considerar:
- No encontamos filas duplicadas
- No encontramos celdas ausentes.
- La Variable Category tiene un único valor. Esto indica que no aporta información relevante y se puede eliminar.
- Las variables Billl_To_Customer y Buying_Group aportan la misma información, por lo que podemos considerar eliminar una de las 2.

### Variable Dimension Date

In [ ]:
datos_dim_date = pd.read_csv('datos_originales/dimension_date.csv', delimiter=',', encoding='latin-1')
datos_dim_date.head()

Veamos el número de filas y columnas que se encuentran en nuestro dataset

In [ ]:
print("Número de filas o datos: {}".format(len(datos_dim_date)))

print("Número de columnas o variables: {}".format(len(datos_dim_date.columns)))

Para ver una exploración más exhaustiva, hicimos un pandas profiling de los datos que nos fueron entregados:

In [ ]:
profiling = ProfileReport(datos_dim_date)
profiling.to_file(output_file="pandas_profiling/dimension_date_pandas_profiling.html")
profiling

Del anterior reporte podemos concluir y considerar:
- No encontamos filas duplicadas
- No encontramos celdas ausentes.
- Las variables Day_Number y Day_val aportan la misma información, por lo que podemos considerar eliminar una de las 2.
- Las variables Month_val y Short_Month aportan la misma información, por lo que podemos considerar eliminar una de las 2.

### Variable Dimension Employee

In [ ]:
datos_dim_emp = pd.read_csv('datos_originales/dimension_employee.csv', delimiter=',', encoding='latin-1')
datos_dim_emp.head()

Vemos que la primera fila, con Employee_Key 0, no aporta información relevante pues todos sus datos están ausentes. Procedemos entonces a eliminarla.

In [ ]:
datos_dim_emp = datos_dim_emp.drop(0)
#Verificamos que se eliminó correctamente
datos_dim_emp.head()

Observamos la cantidad de filas y columnas que se encuentran en nuestro dataset

In [ ]:
print("Número de filas o datos: {}".format(len(datos_dim_emp)))
print("Número de columnas o variables: {}".format(len(datos_dim_emp.columns)))

Para hacer una exploración más exhasutiva, hicimos un pandas profiling de los datos

In [ ]:
profiling = ProfileReport(datos_dim_emp)
profiling.to_file(output_file="pandas_profiling/dimension_employee_pandas_profiling.html")
profiling

Del anterior reporte podemos concluir y considerar:
- No encontamos filas duplicadas
- No encontramos celdas ausentes.
- No se debe realizar ningún paso de preprocesamiento para esta variable

### Variable Dimension Stock Item

In [ ]:
datos_dim_stock = pd.read_csv('datos_originales/dimension_stock_item.csv', delimiter=',', encoding='latin-1')
datos_dim_stock.head()

Vemos que la primera fila, con Item_Key 0, no aporta información relevante pues todos sus datos están ausentes. Procedemos entonces a eliminarla.

In [ ]:
datos_dim_stock = datos_dim_stock.drop(0)
#Verificamos que se eliminó correctamente
datos_dim_stock.head()

Observamos la cantidad de filas y columnas presentes en el dataset

In [ ]:
print("Número de filas o datos: {}".format(len(datos_dim_stock)))
print("Número de columnas o variables: {}".format(len(datos_dim_stock.columns)))

Para hacer una exploración más exhaustiva, hicimos un pandas profiling de los datos

In [ ]:
profiling = ProfileReport(datos_dim_stock)
profiling.to_file(output_file="pandas_profiling/dimension_stock_item_pandas_profiling.html")
profiling

Del anterior reporte podemos concluir y considerar:
- No encontamos filas duplicadas
- Encontramos 1118 celdas vacías que corresponden al 11.1% de los datos. Como el objetivo de este laboratorio no es crear un modelo de ML sino responder a preguntas del negocio sobre sus datos y operación, no podemos reemplazar estos valores vacíos por otros, por lo que los dejaremos como están.
- Observamos que la variable Brand tiene un valor constante en las celdas que tienen un valor asignado, pero además presenta un 90% de ausencias, por lo que no aporta información relevante y podemos considerar eliminarla.

## Preprocesamiento de los datos

### Variable Dimension City

Como vimos en la exploración, las variables row ID y City_Key tienen un valor distinto en cada fila. Eliminamos entonces la variable row ID para dejar City_Key como identificador único. Igualmente eliminamos las variables Country, Continent, Region y Subregion 

In [ ]:
datos_dim_city = datos_dim_city.drop(['row ID', 'Country', "Continent", "Region", "Subregion"], axis=1)

In [ ]:
datos_dim_city.head()

### Variable Dimension Customer

Como vimos en la exploración de la variable Category, ésta tiene un mismo valor en cada celda. Esto indica que no aporta información relevante y procedemos a eliminarla. Igualmente eliminamos la variable Bill_To_Customer para dejar la variable Buying_Group para aportar la misma información

In [ ]:
datos_dim_cust = datos_dim_cust.drop(['Category', 'Bill_To_Customer'], axis=1)

In [ ]:
datos_dim_cust.head()

### Variable Dimension Date

Como vimos en la exploración de las variables Day_Number y Day_val aportan la misma información, por lo que eliminamos Day_val. Lo mismo sucede con Month_val y Short_Month, por lo que eliminamos Short_Month

In [ ]:
datos_dim_date = datos_dim_date.drop(['Day_val', 'Short_Month'], axis=1)

In [ ]:
datos_dim_date.head()

### Variable Dimension Stock Item

Como vimos en el análisis de la variable Brand, ésta no aporta información relevante, por lo que podemos eliminarla.

In [ ]:
datos_dim_stock = datos_dim_stock.drop(['Brand'], axis=1)

In [ ]:
datos_dim_stock.head()